In [ ]:

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.26-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding,LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import wave
import contextlib
import os
import datetime

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.executing_eagerly()

Instructions for updating:
non-resource variables are not supported in the long term


False

In [ ]:

mark_start = 'ssss '
mark_end = ' eeee'

data_src = []
data_dest = []
veri_src = []
veri_dest = []

for line in open('/content/drive/MyDrive/Python/tur.txt', encoding='UTF-8'):
    en_text, tr_text = line.rstrip().split('\t')
    
    tr_text = mark_start + tr_text + mark_end
    
    data_src.append(en_text)
    data_dest.append(tr_text)

    



In [ ]:
class TokenizerWrap(Tokenizer):
    def __init__(self, texts, padding, reverse=False, num_words=None):
        Tokenizer.__init__(self, num_words=num_words)
        self.fit_on_texts(texts)
        self.index_to_word = dict(zip(self.word_index.values(), self.word_index.keys()))
        self.tokens = self.texts_to_sequences(texts)
        
        if reverse:
            self.tokens = [list(reversed(x)) for x in self.tokens]
            truncating = 'pre'
        else:
            truncating = 'post'
        
        self.num_tokens = [len(x) for x in self.tokens]
        self.max_tokens = np.mean(self.num_tokens) + np.std(self.num_tokens)*2
        self.max_tokens = int(self.max_tokens)
        
        self.tokens_pad = pad_sequences(self.tokens,
                                            maxlen=self.max_tokens,
                                            padding=padding,
                                            truncating=truncating)
    def token_to_word(self, token):
        word = ' ' if token==0 else self.index_to_word[token]
        return word
    def tokens_to_string(self, tokens):
        words = [self.index_to_word[token] for token in tokens if token != 0]
        text = ' '.join(words)
        return text
    def text_to_tokens(self, text, padding, reverse=False):
        tokens = self.texts_to_sequences([text])
        tokens = np.array(tokens)
        
        if reverse:
            tokens = np.flip(tokens, axis=1)
            truncating = 'pre'
        else:
            truncating = 'post'
        
        tokens = pad_sequences(tokens,
                               maxlen=self.max_tokens,
                               padding=padding,
                               truncating=truncating)
        return tokens

In [ ]:
tokenizer_src = TokenizerWrap(texts = data_src,
                              padding = 'pre',
                              reverse = True,
                              num_words = None)
tokenizer_dest = TokenizerWrap(texts = data_dest,
                              padding = 'post',
                              reverse = False,
                              num_words = None)


In [ ]:
tokens_src = tokenizer_src.tokens_pad
tokens_dest = tokenizer_dest.tokens_pad

token_start = tokenizer_dest.word_index[mark_start.strip()]
token_end = tokenizer_dest.word_index[mark_end.strip()]

In [ ]:
encoder_input_data = tokens_src
decoder_input_data = tokens_dest[:,:-1]
decoder_output_data = tokens_dest[:,1:]

num_encoder_words = len(tokenizer_src.word_index)+1
num_decoder_words = len(tokenizer_dest.word_index)+1



embedding_size = 100


In [ ]:
word2vec = {}
with open('/content/drive/MyDrive/Colab Notebooks/glove.6B.100d.txt', encoding = 'UTF-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec

embedding_matrix = np.random.uniform(-1, 1, (num_encoder_words, embedding_size))
for word, i in tokenizer_src.word_index.items():
    if i < num_encoder_words:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector 



In [ ]:
encoder_input = Input(shape=(None,), name='encoder_input')

encoder_embedding = Embedding(input_dim=num_encoder_words,
                              output_dim=embedding_size,
                              weights=[embedding_matrix],
                              trainable=True,
                              name='encoder_embedding')
state_size = 256



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
encoder_gru1 = GRU(state_size, name='encoder_gru1', return_sequences=True)
encoder_gru2 = GRU(state_size, name='encoder_gru2', return_sequences=True)
encoder_gru3 = GRU(state_size, name='encoder_gru3', return_sequences=False)




In [ ]:
def connect_encoder():
    net = encoder_input
    
    net = encoder_embedding(net)
    
    net = encoder_gru1(net)
    net = encoder_gru2(net)
    net = encoder_gru3(net)
    
    encoder_output = net
    
    return encoder_output

In [ ]:
encoder_output = connect_encoder()

decoder_initial_state = Input(shape=(state_size,), name='decoder_initial_state')
decoder_input = Input(shape=(None,), name='decoder_input')


decoder_embedding = Embedding(input_dim = num_decoder_words,
                              output_dim = embedding_size,
                              name = 'decoder_embedding')



In [ ]:
decoder_gru1 = GRU(state_size, name='decoder_gru1', return_sequences=True)
decoder_gru2 = GRU(state_size, name='decoder_gru2', return_sequences=True)
decoder_gru3 = GRU(state_size, name='decoder_gru3', return_sequences=True)


decoder_dense = Dense(num_decoder_words,
                      activation='linear',
                      name='decoder_output')



In [ ]:
def connect_decoder(initial_state):
    net = decoder_input
    
    net = decoder_embedding(net)
    
    net = decoder_gru1(net,initial_state=initial_state)
    net = decoder_gru2(net,initial_state=initial_state)
    net = decoder_gru3(net,initial_state=initial_state)

    
    decoder_output = decoder_dense(net)
    
    return decoder_output

In [ ]:
decoder_output = connect_decoder(initial_state = encoder_output)
model_train = Model(inputs=[encoder_input, decoder_input], outputs =[decoder_output])
model_encoder = Model(inputs=[encoder_input], outputs=[encoder_output])
decoder_output1 = connect_decoder(initial_state = decoder_initial_state)
model_decoder = Model(inputs=[decoder_input, decoder_initial_state], outputs=[decoder_output1])



In [ ]:
def sparse_cross_entropy(y_true, y_pred):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    loss_mean = tf.reduce_mean(loss)
    return loss_mean

In [ ]:
optimizer = RMSprop(lr= 1e-3)
decoder_target = tf.placeholder(dtype='int32', shape=(None,None))
model_train.compile(optimizer=optimizer,
                    loss = sparse_cross_entropy,
                    target_tensors =[decoder_target] )




/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
x_data = {'encoder_input': encoder_input_data, 'decoder_input': decoder_input_data}
y_data = {'decoder_output': decoder_output_data}




In [ ]:
path_checkpoint = '/content/drive/MyDrive/Colab Notebooks/Mycheckpoint.keras'
checkpoint = ModelCheckpoint(filepath=path_checkpoint, save_weights_only=True)
try:
    model_train.load_weights(path_checkpoint)
except Exception as error:
    print('Checkpoint yüklenirken hata oluştu. Eğitime sıfırdan başlanıyor.')
    print(error)

In [ ]:
model_train.fit(x=x_data,
                y=y_data,
                batch_size=1024,
                callbacks = [checkpoint],
                epochs=0)

Train on 473035 samples


In [ ]:
mark_start = 'ssss '
mark_end = ' eeee'

veri_src = []
veri_dest = []

for line in open('/content/drive/MyDrive/Python/tur.txt', encoding='UTF-8'):
    en_metin, tr_metin = line.rstrip().split('\t')
    
    en_metin = mark_start + en_metin + mark_end
    
    veri_src.append(tr_metin)
    veri_dest.append(en_metin)

src_tokenizer = TokenizerWrap(texts = veri_src,
                              padding = 'pre',
                              reverse = True,
                              num_words = None)
dest_tokenizer = TokenizerWrap(texts = veri_dest,
                              padding = 'post',
                              reverse = False,
                              num_words = None)

src_tokens = src_tokenizer.tokens_pad
dest_tokens = dest_tokenizer.tokens_pad

token_basla = dest_tokenizer.word_index[mark_start.strip()]
token_bitis = dest_tokenizer.word_index[mark_end.strip()]

encoder_input_veri = src_tokens
decoder_input_veri = dest_tokens[:,:-1]
decoder_output_veri = dest_tokens[:,1:]

num_encoder_word = len(tokenizer_src.word_index)+1
num_decoder_word = len(tokenizer_dest.word_index)+1



embedding_size = 100

encoder_giris = Input(shape=(None,), name='encoder_giris')

encoderEmbedding = Embedding(input_dim = num_encoder_word,
                              output_dim = embedding_size,
                              name = 'encoderEmbedding')

state_size = 256

encoderGRU1 = GRU(state_size, name='encoderGRU1', return_sequences=True)
encoderGRU2 = GRU(state_size, name='encoderGRU2', return_sequences=True)
encoderGRU3 = GRU(state_size, name='encoderGRU3', return_sequences=False)


def connec_encoder():
    net = encoder_giris
    
    net = encoderEmbedding(net)
    
    net = encoderGRU1(net)
    net = encoderGRU2(net)
    net = encoderGRU3(net)
    
    encoder_cikis = net
    
    return encoder_cikis

encoder_cikis = connec_encoder()

decoder_initial_stat = Input(shape=(state_size,), name='decoder_initial_stat')
decoder_giris = Input(shape=(None,), name='decoder_giris')


decoderEmbedding = Embedding(input_dim = num_decoder_word,
                              output_dim = embedding_size,
                              name = 'decoderEmbedding')
    



decoderGRU1 = GRU(state_size, name='decoderGRU1', return_sequences=True)
decoderGRU2 = GRU(state_size, name='decoderGRU2', return_sequences=True)
decoderGRU3 = GRU(state_size, name='decoderGRU3', return_sequences=True)


decoderDense = Dense(num_decoder_word,
                      activation='linear',
                      name='decoder_cikis')

def connec_decoder(initial_state):
    net = decoder_giris
    
    net = decoderEmbedding(net)
    
    net = decoderGRU1(net,initial_state=initial_state)
    net = decoderGRU2(net,initial_state=initial_state)
    net = decoderGRU3(net,initial_state=initial_state)

    
    decoder_cikis = decoderDense(net)
    
    return decoder_cikis

decoder_cikis = connec_decoder(initial_state = encoder_cikis)
tr_model_train = Model(inputs=[encoder_giris, decoder_giris], outputs =[decoder_cikis])
modelEncoder = Model(inputs=[encoder_giris], outputs=[encoder_cikis])
decoder_output_1 = connec_decoder(initial_state = decoder_initial_stat)
tr_model_decoder = Model(inputs=[decoder_giris, decoder_initial_stat], outputs=[decoder_output_1])

tr_model_train.compile(optimizer=optimizer,
                    loss = sparse_cross_entropy,
                    target_tensors =[decoder_target] )

xx_data = {'encoder_giris': encoder_input_veri, 'decoder_giris': decoder_input_veri}
yy_data = {'decoder_cikis': decoder_output_veri}

path_checkpoint = '/content/drive/MyDrive/Colab Notebooks/TrMycheckpoint.keras'
tr_checkpoint = ModelCheckpoint(filepath=path_checkpoint, save_weights_only=True)
try:
    tr_model_train.load_weights(path_checkpoint)
except Exception as error:
    print('Checkpoint yüklenirken hata oluştu. Eğitime sıfırdan başlanıyor.')
    print(error)












In [ ]:
tr_model_train.fit(x=xx_data,
                  y=yy_data,
                  batch_size = 1280,
                  callbacks = [tr_checkpoint],
                  epochs = 0
)

Train on 473035 samples


In [ ]:
def translate(input_text, true_output_text=None):
    input_tokens = tokenizer_src.text_to_tokens(text=input_text,
                                               reverse=True,
                                                padding='pre')
    initial_state = model_encoder.predict(input_tokens)
    max_tokens = tokenizer_dest.max_tokens
    decoder_input_data = np.zeros(shape=(1, max_tokens), dtype=np.int)
    token_int = token_start
    output_text = ''
    count_tokens = 0
    
    while token_int != token_end and count_tokens < max_tokens:
        decoder_input_data[0, count_tokens] = token_int
        x_data = {'decoder_initial_state': initial_state, 'decoder_input': decoder_input_data}
            
        decoder_output = model_decoder.predict(x_data)
        token_onehot = decoder_output[0, count_tokens, :]
        token_int = np.argmax(token_onehot)
            
        sampled_word = tokenizer_dest.token_to_word(token_int)
        output_text += ' ' + sampled_word
        count_tokens += 1
        

        

    return output_text

        
            

In [ ]:
def tr_translate(input_text):
  input_tokens = src_tokenizer.text_to_tokens(text=input_text,
                                               reverse=True,
                                                padding='pre')
  initialState = modelEncoder.predict(input_tokens)
  max_tokens = dest_tokenizer.max_tokens
  decoder_input_data = np.zeros(shape=(1, max_tokens), dtype=np.int)
  token_int = token_basla
  metin = ''
  count_tokens = 0

  while token_int != token_bitis and count_tokens < max_tokens:
    decoder_input_veri[0, count_tokens] = token_int
    xx_data = {'decoder_initial_stat': initialState, 'decoder_giris' : decoder_input_data}

    decoder_cikis = tr_model_decoder.predict(xx_data)
    token_onehot = decoder_cikis[0, count_tokens, :]
    token_int = np.argmax(token_onehot)

    sampled_word = dest_tokenizer.token_to_word(token_int)
    metin += ' ' + sampled_word
    count_tokens += 1

    return metin


  


In [ ]:
pip install SpeechRecognition

     |████████████████████████████████| 32.8MB 89kB/s 


In [ ]:
pip install pydub

In [ ]:
import speech_recognition as sr 
from pydub import AudioSegment 
from pydub.silence import split_on_silence 
import io  
import os 
r = sr.Recognizer()
utku_can = []
def audio_to_text(file):
    sound = AudioSegment.from_mp3(file)
    chunks = split_on_silence(sound,min_silence_len=500,
                              silence_thresh=sound.dBFS-14,
                              keep_silence=500,
    )
    
    folder_name = "/content/drive/MyDrive/audio-chunk"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                text="12.<"
                whole_text += text
            else:
                text = f"{text.capitalize()}.< "
                whole_text += text
                            
    return whole_text






 


Royal armouries.< 12.<


In [ ]:
def audio_to_tr_text(file):
    sound = AudioSegment.from_mp3(file)
    chunks = split_on_silence(sound,min_silence_len=500,
                              silence_thresh=sound.dBFS-14,
                              keep_silence=500,
    )
    
    folder_name = "/content/drive/MyDrive/audio-chunk"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            try:
                text = r.recognize_google(audio_listened, language='tr-TR')
            except sr.UnknownValueError as e:
                text="12.<"
                whole_text += text
            else:
                text = f"{text.capitalize()}.< "
                whole_text += text
                            
    return whole_text

   


O yaramaz olsun portakal seçmeye gelmedim dayı cevap almaya geldim ne hakkında hapis ömer kerem adım asap bozma değil gel güzel güzel konuşalım bizim zamanımız delikanlı adam yapacağını sokak ortasında yapardı kardeş sizin zamandan kimse kalmadı hepsini temizledik ben kaldım ben kaldım yok dayı yanlışımla sen de gittin hikayen kaldı sadece artık çağırınca tıpış tıpış geleceksin anlatacak mısın kardeş yaprağından hafızada zayıfladı birazdan senin burada olduğunu bile unuturum bakarsın ben seni unutacağım adamlardan değilim gözlerimin içine bak ramiz anladın mı nasıl bir adam bilemem kardeş meyveyi soymadan içinden ne çıkacak bilemem sana beri anlatayım.< Bayan arkadaş şeytan zamanında değil mi ömer bugün burada senin canını kurtaran adam senin hayatını kim kurtaracak ramiz çıkacaksın o zaman ne olacak o da güzel hikaye ama erken anlatılırsa tadı kaçar önce tecrübe etmen gerek kardeş.< 


In [ ]:
from google.colab import files 
import moviepy.editor as mp
import ipywidgets as widgets
from IPython.display import display

#@title Otomatik Altyazı Oluşturma Sistemine Hoşgeldiniz...
#@markdown ### Lütfen Altyazı Oluşturmak İstediğiniz Seçeneği Seçiniz...
Seçim = "Yabanc\u0131 Dizi" #@param ["Yerli Dizi","Yabancı Dizi"]
#@markdown Lütffen Altyazı Oluşturmak İstediğiniz MP4 Dosyasını Seçiniz...
uploaded = files.upload()

for fn in uploaded.keys():
  print('"{name}" Dosyası Yüklendi.\n Çeviriye Başlanıyor...'.format(
      name=fn))
clip = mp.VideoFileClip(fn)
fn=fn[:-1]
fn = fn + '3'
clip.audio.write_audiofile(fn)
clip = str(clip)


liste = []  
if Seçim == 'Yabanc\u0131 Dizi' :
  for i in range(len_1):
   dizi = audio_to_text(fn)
   ceviri = dizi.split('<')
   len_1 = len(ceviri)
   söz = ceviri[i]
   söz_1 = translate(input_text=söz)
   liste.append(söz_1)

if Seçim == 'Yerli Dizi' :
  for i in range(len_1):
   dizi = audio_to_tr_text(fn)
   ceviri = dizi.split('<')
   len_1 = len(ceviri)
   söz = ceviri[i]
   söz_1 = tr_translate(input_text=söz)
   liste.append(söz_1)

liste_len = len(liste)
cevrilmis_cumle = []
for i in range(liste_len):
  text = liste[i]
  text_1 = ' '.join(word for word in text.split()[:-1])
  cevrilmis_cumle.append(text_1)
cevrilmis_cumle.pop()

for i in range(len(cevrilmis_cumle)):
  if cevrilmis_cumle[i] == 'okula':
    cevrilmis_cumle[i] = ' '
  elif cevrilmis_cumle[i] == '':
    cevrilmis_cumle[i] = ceviri[i] 

video_lenght=[]
i=1
while os.path.exists('/content/drive/MyDrive/audio-chunk'+ fn + '/chunk%s.wav' % i):
    with contextlib.closing(wave.open('/content/drive/MyDrive/audio-chunk'+ fn +'/chunk%s.wav' % i, 'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        duration = frames / float(rate)
        duration = str(datetime.timedelta(seconds=duration))
        video_lenght.append(duration)
        i += 1




end = []
video = []
video_len = []
date_time = '00:00:00.000'
x=0
while x<len(video_lenght):
    date_time2 = video_lenght[x]
    time1 = datetime.datetime.strptime(date_time, '%H:%M:%S.%f')
    time2 = datetime.datetime.strptime(date_time2, '%H:%M:%S.%f')
    time_zero = datetime.datetime.strptime('00:00:00.000', '%H:%M:%S.%f')
    date_time = (time1 - time_zero + time2).time()
    date_time = str(date_time)
    video.append(date_time)
    x += 1

begin = ['00:00:00,000']
a = 0


u=0
while u<len(video):
  utku = video[u]
  utku = utku[:-3]
  video_len.append(utku)
  u += 1



k = 0

while k<len(video_len):
  rep = video_len[k]
  rep = rep.replace('.',',')
  end.append(rep)
  k+=1
 


while a<len(end) - 1 :
  begin.append(end[a])
  a += 1   
u=1
w=0
fn = fn[:-4]
while w<len(cevrilmis_cumle):
  with open("/content/drive/MyDrive/Python/"+fn+".srt", "a") as f:
    f.write(str(u) + "\n" + begin[w] + " --> " + end[w] + "\n" +
            cevrilmis_cumle[w] + "\n" + '\n')
    f.close
    u += 1
    w +=1

print("İndirme İşlemi Başlatılıyor...")
files.download("/content/drive/MyDrive/Python/"+fn+".srt")
print("İndirme İşlemi Başarılı\nYine Bekleriz...")






Saving underrated funny moments in avatar the last airbender pt 1.mp4 to underrated funny moments in avatar the last airbender pt 1.mp4
"underrated funny moments in avatar the last airbender pt 1.mp4" Dosyası Yüklendi.
 Çeviriye Başlanıyor...
[MoviePy] Writing audio in underrated funny moments in avatar the last airbender pt 1.mp3


100%|██████████| 7583/7583 [00:09<00:00, 783.53it/s]

[MoviePy] Done.


İndirme İşlemi Başlatılıyor...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

İndirme İşlemi Başarılı
Yine Bekleriz...
